# Optimizing Neural Network

## Preprocessing

### Imports

In [8]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
#Comment back in on Collab
#import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Pipelines

Original

In [25]:
def default_pipeline(application_df):
    
    # Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
    application_df = application_df.drop(labels=["EIN", "NAME"], axis=1)

    # Bucket APPLICATION_TYPE
    application_counts = application_df["APPLICATION_TYPE"].value_counts()

    # Determine which values to replace if counts are less than 500
    replace_application = list(application_counts[application_counts < 500].index)

    # Replace in dataframe
    for app in replace_application:
        application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

    # Bucket CLASSIFICATION
    classification_counts = application_df["CLASSIFICATION"].value_counts()

    # Determine which values to replace if counts are less than 1800
    replace_class = list(classification_counts[classification_counts < 1800].index)

    # Replace in dataframe
    for cls in replace_class:
        application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")

    # Generate our categorical variable lists
    application_cat = list(application_df.dtypes[application_df.dtypes == "object"].index)

    # Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

    # Add the encoded variable names to the dataframe
    encode_df.columns = enc.get_feature_names(application_cat)

    # Merge one-hot encoded features and drop the originals
    application_df = application_df.merge(encode_df, left_index=True, right_index=True)
    application_df = application_df.drop(labels=application_cat, axis=1)
    return application_df    

Reduced Bins

In [34]:
def reduced_pipeline(application_df):
    
    # Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
    application_df = application_df.drop(labels=["EIN", "NAME"], axis=1)

    # Bucket APPLICATION_TYPE
    application_counts = application_df["APPLICATION_TYPE"].value_counts()

    # Determine which values to replace if counts are less than 500
    replace_application = list(application_counts[application_counts < 500].index)

    # Replace in dataframe
    for app in replace_application:
        application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

    # Bucket CLASSIFICATION
    classification_counts = application_df["CLASSIFICATION"].value_counts()

    # Determine which values to replace if counts are less than 1800
    replace_class = list(classification_counts[classification_counts < 1800].index)

    # Replace in dataframe
    for cls in replace_class:
        application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")
        
    income_counts = application_df["INCOME_AMT"].value_counts()
        
    # Determine which values to replace if counts are less than 3000
    replace_income = list(income_counts[income_counts < 3000].index)

    # Replace in dataframe
    for income in replace_income:
        application_df["INCOME_AMT"] = application_df["INCOME_AMT"].replace(income, "Other")
    
    aff_counts = application_df["AFFILIATION"].value_counts()
    
    # Determine which values to replace if counts are less than 15000
    replace_aff = list(aff_counts[aff_counts < 15000].index)

    # Replace in dataframe
    for aff in replace_aff:
        application_df["AFFILIATION"] = application_df["AFFILIATION"].replace(aff, "Other")

    # Generate our categorical variable lists
    application_cat = list(application_df.dtypes[application_df.dtypes == "object"].index)

    # Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

    # Add the encoded variable names to the dataframe
    encode_df.columns = enc.get_feature_names(application_cat)

    # Merge one-hot encoded features and drop the originals
    application_df = application_df.merge(encode_df, left_index=True, right_index=True)
    application_df = application_df.drop(labels=application_cat, axis=1)
    
    # Drop "SPECIAL_CONSIDERATION_N":
    application_df = application_df.drop("SPECIAL_CONSIDERATIONS_N", axis=1)
    return application_df    

In [35]:
# instantiating df's for pipelines above
df = default_pipeline(application_df)
df2 = reduced_pipeline(application_df)

### Reset dataset function (_for new testing_)

In [16]:
def new_dataset(df):
    # Split our preprocessed data into our features and target arrays
    y = df["IS_SUCCESSFUL"].values.reshape(-1, 1)
    X = df.drop("IS_SUCCESSFUL", axis=1).values

    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Create a StandardScaler instances
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train = X_scaler.transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

In [17]:
X_train, X_test, y_train, y_test = new_dataset(df)

## Compile, Train and Evaluate the Models

### define models for quick boot

In [37]:
def sig_model(X_train_scaled):
    # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
    number_input_features = len(X_train_scaled[0])

    # number of layer1 neurons = 2*(number of inputs=43) = 86 ~ 80
    hidden_nodes_layer1 = 80

    # number of layer2 neurons: Between (input=80) and (output=1 - classifier)
    hidden_nodes_layer2 = 30

    nn_inc_epochs = tf.keras.models.Sequential()

    # First hidden layer
    nn_inc_epochs.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn_inc_epochs.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

    # Output layer
    nn_inc_epochs.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Check the structure of the model
    return nn_inc_epochs.summary()

In [38]:
def tanh_model(X_train_scaled):
    # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
    number_input_features = len(X_train_scaled[0])

    # number of layer1 neurons = 2*(number of inputs=43) = 86 ~ 80
    hidden_nodes_layer1 = 80

    # number of layer2 neurons: Between (input=80) and (output=1 - classifier)
    hidden_nodes_layer2 = 30

    nn_inc_epochs = tf.keras.models.Sequential()

    # First hidden layer
    nn_inc_epochs.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn_inc_epochs.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

    # Output layer
    nn_inc_epochs.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Check the structure of the model
    return nn_inc_epochs.summary()

# relu-sig model with variations

In [39]:
X_train, X_test, y_train, y_test = new_dataset(df)
sig = sig_model(X_train)

In [ ]:
# Compile the model
sig.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model_sig = sig.fit(X_train, y_train, epochs=70)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = sig.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
X_train, X_test, y_train, y_test = new_dataset(df)
tanh = tanh_model(X_train)

In [34]:
# Compile the model
tanh.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model_tanh = tanh.fit(X_train, y_train, epochs=90)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = tanh.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [38]:
# Export and save model to HDF5 file
tanh.save("AlphabetSoupCharity_Optimization.h5")